<a href="https://colab.research.google.com/github/Matthew0077/genai/blob/main/%E7%84%A6%E6%85%AE%E6%83%85%E7%B7%92%E8%AA%BF%E7%AF%80_%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import ipywidgets as widgets
widgets.Textarea()

Textarea(value='')

In [ ]:
from google.colab import userdata
print("你的OpenAI密鑰：", userdata.get('OpenAI'))

In [ ]:
from google.colab import userdata
import openai

openai.api_key = userdata.get('OpenAI')
print("API Key 設定完成？", bool(openai.api_key))

API Key 設定完成？ True


In [ ]:
from google.colab import userdata
import openai

api_key = userdata.get('OpenAI')
print("API Key:", api_key)   # 確認不是 None
openai.api_key = api_key
print("API Key 設定完成？", bool(openai.api_key))  # 必須是 True

### 上面是debug用的

In [ ]:
!pip install --upgrade openai ipywidgets sqlalchemy pandas matplotlib
!apt-get -y install fonts-noto-cjk
###按取消!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    U

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  fonts-noto-cjk-extra
The following NEW packages will be installed:
  fonts-noto-cjk
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 61.2 MB of archives.
After this operation, 93.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-noto-cjk all 1:20220127+repack1-1 [61.2 MB]
Fetched 61.2 MB in 2s (30.9 MB/s)
Selecting previously unselected package fonts-noto-cjk.
(Reading database ... 126109 files and directories currently installed.)
Preparing to unpack .../fonts-noto-cjk_1%3a20220127+repack1-1_all.deb ...
Unpacking fonts-noto-cjk (1:20220127+repack1-1) ...
Setting up fonts-noto-cjk (1:20220127+repack1-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Noto Sans CJK TC'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Noto Sans CJK TC'
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
import sqlite3
import openai
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# 初始化 SQLite 資料庫
conn = sqlite3.connect('anxiety_journal.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS entries (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    created_at TEXT NOT NULL,
    content TEXT NOT NULL,
    anxiety_score REAL,
    ai_feedback TEXT
)
''')
conn.commit()

# GPT-4 分析日記
def analyze_entry(text: str) -> tuple[float, str]:
    prompt = f"""
你是一位教育心理學專家，請根據以下日記文字進行分析：
---
{text}
---
1. 以 0 到 100 的尺度給出「焦慮指數」，0 表示毫無焦慮，100 表示極度焦慮。
2. 給出一段 2～3 句的「調節建議」。
請用這個格式回覆：
焦慮指數：<數值>
調節建議：<文字>
"""
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "你是教育心理學專家。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=150
    )
    reply = response.choices[0].message.content.strip()
    score = 0.0
    feedback = ""
    for line in reply.splitlines():
        if line.startswith("焦慮指數"):
            try:
                score = float(line.split("：")[1].strip())
            except:
                score = 0.0
        elif line.startswith("調節建議"):
            feedback = line.split("：", 1)[1].strip()
    return score, feedback

# 設計 widgets 元件
textarea = widgets.Textarea(
    value='',
    placeholder='請在此輸入今天的焦慮日記…',
    description='日記輸入：',
    layout=widgets.Layout(width='80%', height='120px')
)
submit_btn = widgets.Button(
    description='送出並分析',
    button_style='primary'
)
out_area = widgets.Output()

def on_submit_clicked(b):
    with out_area:
        out_area.clear_output()
        user_text = textarea.value.strip()
        if not user_text:
            print("❗ 請先輸入日記內容。")
            return
        score, feedback = analyze_entry(user_text)
        cursor.execute(
            'INSERT INTO entries (created_at, content, anxiety_score, ai_feedback) VALUES (?, ?, ?, ?)',
            (datetime.utcnow().strftime("%Y-%m-%d %H:%M"), user_text, score, feedback)
        )
        conn.commit()
        print(f"✅ 已儲存。焦慮指數：{score:.1f}")
        print(f"🤖 AI 建議：{feedback}\n")
        df_all = pd.read_sql_query('SELECT * FROM entries ORDER BY created_at ASC', conn)
        print("--- 歷史所有日記紀錄 ---")
        display(df_all)
        if len(df_all) > 1:
            df_all['created_at'] = pd.to_datetime(df_all['created_at'])
            plt.figure(figsize=(7, 3))
            plt.plot(df_all['created_at'], df_all['anxiety_score'], marker='o', linestyle='-', color='tab:blue')
            plt.xlabel("日期")
            plt.ylabel("焦慮指數")
            plt.title("焦慮指數折線圖（歷史）")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.grid(alpha=0.3)
            plt.show()
        else:
            print("（再多寫幾天日記，就會自動產生折線圖啦！）")

submit_btn.on_click(on_submit_clicked)

# 顯示 widgets 介面
display(textarea, submit_btn, out_area)


Textarea(value='', description='日記輸入：', layout=Layout(height='120px', width='80%'), placeholder='請在此輸入今天的焦慮日記……

Button(button_style='primary', description='送出並分析', style=ButtonStyle())

Output()

### 等待30秒